In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from merge_years.import_data import get_full_data

# Data

In [2]:
raw_data = get_full_data('../raw_data')

## Data split

In [3]:
# Filling the Caglar's position

j=0
for i,row in raw_data.iterrows():
    if row['name']=='caglar söyüncü':
        print(j)
        raw_data['position'].at[i]='DEF'
        j+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [4]:
raw_data['selected'].unique() # done

array([     0,  14029,  22804, ..., 275199, 273494, 270474])

In [5]:
# train and test split 
ra = 2

test_df = raw_data[(raw_data.season > 20) & (raw_data.GW >= 29-ra)]
train_df = raw_data[~((raw_data.season > 20) & (raw_data.GW >= 29-ra))]

test_df.shape, train_df.shape

((8029, 40), (106963, 40))

In [7]:
#train_df =  train_df[train_df['minutes'] > 0]

#train_df =  train_df[train_df['name'] == ]


## Data preprocessing

In [8]:
# Train and test sets with interesting features

train_df = train_df.drop(columns=['dreamteam_yearly_average', 'team_level', 'opponent_level', 
                                   'team_name', 'team_id', 'fixture', 'element', 'team_a_score', 
                                   'team_h_score', 'transfers_in', 'transfers_out', 'kickoff_time', 
                                   'kickoff_date', 'opponent_team', 'influence', 'creativity', 'threat', 
                                  'transfers_balance']).copy()

test_df = test_df.drop(columns=['dreamteam_yearly_average', 'team_level', 'opponent_level', 
                                 'team_name', 'team_id', 'fixture', 'element', 'team_a_score', 
                                 'team_h_score', 'transfers_in', 'transfers_out', 'kickoff_time', 
                                 'kickoff_date', 'opponent_team', 'influence', 'creativity', 'threat', 
                                'transfers_balance']).copy()

# 27 features left -> drop selected?

In [9]:
#train_df['selected'].unique()

In [10]:
train_df.head()

,name,assists,bonus,bps,clean_sheets,goals_conceded,goals_scored,ict_index,minutes,own_goals,...,red_cards,saves,selected,total_points,value,was_home,yellow_cards,GW,season,position
0,aaron connolly,0,0,1,0,1,0,0.0,24,0,...,0,0,0,1,45,False,0,4,20,FWD
1,aaron connolly,0,0,1,0,1,0,2.2,6,0,...,0,0,14029,1,45,True,0,5,20,FWD
2,aaron connolly,0,0,1,0,0,0,2.5,20,0,...,0,0,22804,1,45,False,0,6,20,FWD
3,aaron connolly,0,0,2,0,1,0,0.1,25,0,...,0,0,32699,1,45,False,0,7,20,FWD
4,aaron connolly,0,3,53,1,0,2,20.1,79,0,...,0,0,35026,13,45,True,0,8,20,FWD


### Rolling Average (ra)

In [11]:
# Features to include in ra (keep name)

rolling_train_df = train_df.drop(columns=['was_home', 'GW', 'season', 'position'])
rolling_test_df = test_df.drop(columns=['was_home', 'GW', 'season', 'position'])

# Ra by name

rolling_train_df = rolling_train_df.groupby('name').rolling(ra, closed = 'left').mean()
rolling_test_df = rolling_test_df.groupby('name').rolling(ra, closed = 'left').mean()

# Creating a rolling points column

rolling_train_df.rename(columns={'total_points':'rolling_points'},inplace=True)
rolling_test_df.rename(columns={'total_points':'rolling_points'},inplace=True)

# Reinserting a total points column (not rolled)

total_points_train = train_df['total_points']
total_points_test = test_df['total_points']

result = map(lambda position:position, total_points_train)
rolling_train_df['total_points']=list(result)
result = map(lambda position:position, total_points_test)
rolling_test_df['total_points']=list(result)

# Reset index

train_df_ra = rolling_train_df.reset_index()
test_df_ra = rolling_test_df.reset_index()

In [12]:
# adding to train: was_home, GW, season, position

result = map(lambda gw:gw, train_df['GW'])
train_df_ra['GW'] = list(result)

result_1 = map(lambda season:season, train_df['season'])
train_df_ra['season'] = list(result_1)

result_2 = map(lambda home:home, train_df['was_home'])
train_df_ra['was_home'] = list(result_2)

result_5 = map(lambda position:position, train_df['position'])
train_df_ra['position'] = list(result_5)

# adding test: was_home, GW, season, position 

result = map(lambda gw:gw, test_df['GW'])
test_df_ra['GW'] = list(result)

result_1 = map(lambda season:season, test_df['season'])
test_df_ra['season'] = list(result_1)

result_2 = map(lambda home:home, test_df['was_home'])
test_df_ra['was_home'] = list(result_2)

result_5 = map(lambda position:position, test_df['position'])
test_df_ra['position'] = list(result_5)


In [13]:
# Drop col: level_1 and nans

train_df_ra.drop(columns=['level_1'], inplace=True)
test_df_ra.drop(columns=['level_1'], inplace=True)

train_preproc = train_df_ra.dropna()
test_preproc = test_df_ra.dropna()

In [14]:
train_preproc.head()

,name,assists,bonus,bps,clean_sheets,goals_conceded,goals_scored,ict_index,minutes,own_goals,...,saves,selected,rolling_points,value,yellow_cards,total_points,GW,season,was_home,position
2,aaron connolly,0.0,0.0,1.0,0.0,1.0,0.0,1.10,15.0,0.0,...,0.0,7014.5,1.0,45.0,0.0,1,6,20,False,FWD
3,aaron connolly,0.0,0.0,1.0,0.0,0.5,0.0,2.35,13.0,0.0,...,0.0,18416.5,1.0,45.0,0.0,1,7,20,False,FWD
4,aaron connolly,0.0,0.0,1.5,0.0,0.5,0.0,1.30,22.5,0.0,...,0.0,27751.5,1.0,45.0,0.0,13,8,20,True,FWD
5,aaron connolly,0.0,1.5,27.5,0.5,0.5,1.0,10.10,52.0,0.0,...,0.0,33862.5,7.0,45.0,0.0,1,9,20,False,FWD
6,aaron connolly,0.0,1.5,28.0,0.5,0.5,1.0,11.85,62.0,0.0,...,0.0,72985.0,7.0,45.0,0.0,8,10,20,True,FWD


### Encoding

In [15]:
from sklearn.preprocessing import LabelEncoder

# Encoding was_home feature
# Train:

encoder = LabelEncoder()
encoder.fit(train_preproc[['was_home']])

train_preproc['was_home'] = encoder.transform(train_preproc[['was_home']])

# Test:

encoder_2 = LabelEncoder()
encoder_2.fit(test_preproc[['was_home']])

test_preproc['was_home'] = encoder_2.transform(test_preproc[['was_home']])

/Users/Stemitsiotis/.pyenv/versions/3.8.6/envs/fflpred/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
<ipython-input-15-8730b044da5e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_preproc['was_home'] = encoder.transform(train_preproc[['was_home']])
<ipython-input-15-8730b044da5e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [16]:
from sklearn.preprocessing import OneHotEncoder

# Encoding position
# Train:

ohe = OneHotEncoder(sparse=False)
ohe.fit(train_preproc[['position']])

position_encoded = ohe.transform(train_preproc[['position']])

train_preproc["FWD"],train_preproc["DEF"],train_preproc["MID"],train_preproc["GK"]= position_encoded.T

# Test:

ohe_2 = OneHotEncoder(sparse=False)
ohe_2.fit(test_preproc[['position']])

position_encoded_2 = ohe_2.transform(test_preproc[['position']])

test_preproc["FWD"],test_preproc["DEF"],test_preproc["MID"],test_preproc["GK"]= position_encoded_2.T

<ipython-input-16-ef2bd9b6e34b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_preproc["FWD"],train_preproc["DEF"],train_preproc["MID"],train_preproc["GK"]= position_encoded.T
<ipython-input-16-ef2bd9b6e34b>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_preproc["FWD"],test_preproc["DEF"],test_preproc["MID"],test_preproc["GK"]= position_encoded_2.T


In [17]:
train_preproc = train_preproc.drop(columns=['position'])
test_preproc = test_preproc.drop(columns=['position'])

In [18]:
train_preproc.head()

,name,assists,bonus,bps,clean_sheets,goals_conceded,goals_scored,ict_index,minutes,own_goals,...,value,yellow_cards,total_points,GW,season,was_home,FWD,DEF,MID,GK
2,aaron connolly,0.0,0.0,1.0,0.0,1.0,0.0,1.10,15.0,0.0,...,45.0,0.0,1,6,20,0,0.0,1.0,0.0,0.0
3,aaron connolly,0.0,0.0,1.0,0.0,0.5,0.0,2.35,13.0,0.0,...,45.0,0.0,1,7,20,0,0.0,1.0,0.0,0.0
4,aaron connolly,0.0,0.0,1.5,0.0,0.5,0.0,1.30,22.5,0.0,...,45.0,0.0,13,8,20,1,0.0,1.0,0.0,0.0
5,aaron connolly,0.0,1.5,27.5,0.5,0.5,1.0,10.10,52.0,0.0,...,45.0,0.0,1,9,20,0,0.0,1.0,0.0,0.0
6,aaron connolly,0.0,1.5,28.0,0.5,0.5,1.0,11.85,62.0,0.0,...,45.0,0.0,8,10,20,1,0.0,1.0,0.0,0.0


## Train and Test

In [19]:
X_train = train_preproc.drop(columns=['total_points', 'name'])
y_train = train_preproc['total_points']

X_test = test_preproc.drop(columns=['total_points', 'name'])
y_test = test_preproc['total_points']

In [20]:
X_train_scale = X_train.drop(columns=['GW', 'season', 'was_home', 'FWD', 'DEF', 'MID', 'GK'])
X_test_scale = X_test.drop(columns=['GW', 'season', 'was_home', 'FWD', 'DEF', 'MID', 'GK'])

In [21]:
from sklearn.preprocessing import MinMaxScaler

# MinMax scaling numerical features

scaler = MinMaxScaler()
scaler.fit(X_train_scale)

# Transform X_train

X_train_final = X_train[['GW', 'season', 'was_home', 'FWD', 'DEF', 'MID', 'GK']].reset_index(drop = True)\
        .join(pd.DataFrame(scaler.transform(X_train_scale),
             columns=X_train_scale.columns))

In [22]:
# Transform X_test

X_test_final = X_test[['GW', 'season', 'was_home', 'FWD', 'DEF', 'MID', 'GK']].reset_index(drop = True)\
        .join(pd.DataFrame(scaler.transform(X_test_scale),
             columns=X_test_scale.columns))

## KNN Model

In [23]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

knn_regressor = KNeighborsRegressor(n_neighbors=23).fit(X_train_final, y_train)

knn_score = cross_val_score(knn_regressor, X_train_final, y_train).mean()
knn_score

0.21659899058974616

In [24]:
from sklearn.metrics import mean_absolute_error

y_pred = knn_regressor.predict(X_test_final)

mean_absolute_error(y_test,y_pred)

1.0772048186433099

In [ ]:
knn_regressor.kneighbors(X=X_test_final, n_neighbors=23, return_distance=True)